### Imports

In [87]:
import pandas as pd
from datetime import datetime, timedelta

from transform_ps.transform_ps.donors import Donors

### Get Data

In [88]:
d = Donors(file_yr='13')
d.filter_fys(fys=[20], inplace=True)

### Dates

In [89]:
quarter_start = '2019-10-01'
quarter_end = '2019-12-31'

In [90]:
today_date = datetime.today().strftime("%b-%d-%Y")
last_week_date = datetime.today() - timedelta(days=7)

### Interim dataframes

#### Prior Week

In [91]:
prior_week = d.data[d.data['cont_dt'] <= last_week_date]
prior_week_campaign = prior_week.groupby('campaign').agg({'gift_plus_pledge': 'sum'}).reset_index()
prior_week_solicitor = prior_week.groupby('ps_sol').agg({'gift_plus_pledge': 'sum'}).reset_index()

#### Quarter

In [92]:
q_columns = ['campaign', 'quarter_total']

quarterly = d.data[d.data['cont_dt'].between(quarter_start, quarter_end)]

quarterly_campaign = quarterly.groupby('campaign').agg({'gift_plus_pledge': 'sum'}).reset_index()
quarterly_campaign.columns = q_columns

quarterly_solicitor = quarterly.groupby('ps_sol').agg({'gift_plus_pledge': 'sum'}).reset_index()
quarterly_solicitor.columns = q_columns

#### Current Week

In [93]:
cw_columns = ['campaign', 'to_date']

campaign_current = d.data.groupby('campaign').agg({'gift_plus_pledge': 'sum'}).reset_index()
campaign_current.columns = cw_columns

solicitor_current = d.data.groupby('ps_sol').agg({'gift_plus_pledge': 'sum'}).reset_index()
solicitor_current.columns = cw_columns

### Filling out Columns

#### Campaign Summary

In [94]:
campaign_current['new'] = campaign_current['to_date'] - prior_week_campaign['gift_plus_pledge']
campaign_summary = campaign_current.merge(quarterly_campaign, on='campaign', how='left').fillna(0)

#### Solicitor Summary

In [95]:
solicitor_current['new'] = solicitor_current['to_date'] - prior_week_solicitor['gift_plus_pledge']
solicitor_summary = solicitor_current.merge(quarterly_solicitor, on='campaign', how='left').fillna(0)

### Format

In [41]:
def frmt(amount):
    return round(amount)

In [42]:
campaign_summary['to_date'] = campaign_summary['to_date'].apply(frmt)
campaign_summary['new'] = campaign_summary['new'].apply(frmt)
campaign_summary['quarter_total'] = campaign_summary['quarter_total'].apply(frmt)

solicitor_summary['to_date'] = solicitor_summary['to_date'].apply(frmt)
solicitor_summary['new'] = solicitor_summary['new'].apply(frmt)
solicitor_summary['quarter_total'] = solicitor_summary['quarter_total'].apply(frmt)

### Write to Excel

In [43]:
with pd.ExcelWriter(f'dev_summary_{today_date}.xlsx') as writer:
    campaign_summary.to_excel(writer, sheet_name='campaign_summary', index=False)
    solicitor_summary.to_excel(writer, sheet_name='solicitor_summary', index=False)